In [1]:
import os
import math

import skimage.feature
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from sklearn.cluster import KMeans

/Users/aura_dev/.virtualenvs/w281/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# process function
def process_ellipse(fname):
    # read input
    img = cv2.imread(fname)

    # convert to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # threshold
    thresh = cv2.threshold(gray, 100 , 255, cv2.THRESH_BINARY)[1]

    # find largest contour
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    big_contour = max(contours, key=cv2.contourArea)

    # fit contour to ellipse and get ellipse center, minor and major diameters and angle in degree 
    ellipse = cv2.fitEllipse(big_contour)
    (xc,yc),(d1,d2),angle = ellipse

    # draw ellipse
    result = img.copy()
    cv2.ellipse(result, ellipse, (0, 255, 0), 3)

    # draw circle at center
    xc, yc = ellipse[0]
    cv2.circle(result, (int(xc),int(yc)), 10, (255, 0, 0), 20)

    # draw vertical line
    # compute major radius
    rmajor = max(d1,d2)/2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    xtop_major = xc + math.cos(math.radians(angle))*rmajor
    ytop_major = yc + math.sin(math.radians(angle))*rmajor
    xbot_major = xc + math.cos(math.radians(angle+180))*rmajor
    ybot_major = yc + math.sin(math.radians(angle+180))*rmajor
    cv2.line(result, (int(xtop_major),int(ytop_major)), (int(xbot_major),int(ybot_major)), (0, 0, 255), 3)
    major_distance = ((xtop_major - xbot_major)**2 + (ytop_major - ybot_major)**2)**0.5

    # draw vertical line
    # compute minor radius
    rminor = min(d1,d2)/2
    if angle > 90:
        angle = angle - 90
    else:
        angle = angle + 90

    xtop_minor = xc + math.cos(math.radians(angle))*rminor
    ytop_minor = yc + math.sin(math.radians(angle))*rminor
    xbot_minor = xc + math.cos(math.radians(angle+180))*rminor
    ybot_minor = yc + math.sin(math.radians(angle+180))*rminor
    cv2.line(result, (int(xtop_minor),int(ytop_minor)), (int(xbot_minor),int(ybot_minor)), (0, 0, 255), 3)
    minor_distance = ((xtop_minor - xbot_minor)**2 + (ytop_minor - ybot_minor)**2)**0.5
    
    major_radius = major_distance/2
    minor_radius = minor_distance/2
    ellipse_area = np.pi * major_radius * minor_radius
    ellipse_perimeter = 2 * np.pi * ((major_radius ** 2 + minor_radius ** 2)/2) ** 0.5
    cv2.imwrite(fname.replace("/", "_processed/"), result)
    
    cnt = contours[0]
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt,True)

    rgb_im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv_im = cv2.cvtColor(rgb_im, cv2.COLOR_RGB2HSV)

    pistachio_cords = np.argwhere(gray > 0)
    pistachio_pixs = rgb_im[pistachio_cords[:, 0], pistachio_cords[:, 1]]
    pistachio_hsv = hsv_im[pistachio_cords[:, 0], pistachio_cords[:, 1]]

    X = pistachio_hsv.reshape(-1, 3)
    kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
    results = kmeans.labels_

    unique, counts = np.unique(results, return_counts=True)
    argsort = unique[np.argsort(counts)]
    shell = unique[argsort[2]]
    shell_count = counts[argsort[2]]
    nut = unique[argsort[1]]
    nut_count = counts[argsort[1]]

    shell_cords = pistachio_cords[np.argwhere(results == shell).flatten()]
    nut_cords = pistachio_cords[np.argwhere(results == nut).flatten()]

    shell_pixs = rgb_im[shell_cords[:, 0], shell_cords[:, 1]]
    shell_hsv = hsv_im[shell_cords[:, 0], shell_cords[:, 1]]
    nut_pixs = rgb_im[nut_cords[:, 0], nut_cords[:, 1]]
    nut_hsv = hsv_im[nut_cords[:, 0], nut_cords[:, 1]]

    shell_pixs_mean, nut_pixs_mean, shell_pixs_med, nut_pixs_med, \
    shell_r_mean, shell_g_mean, shell_b_mean, \
    nut_r_mean, nut_g_mean, nut_b_mean, \
    shell_r_med, shell_g_med, shell_b_med, \
    nut_r_med, nut_g_med, nut_b_med, \
    shell_h_mean, shell_s_mean, shell_v_mean, \
    nut_h_mean, nut_s_mean, nut_v_mean, \
    shell_h_median, shell_s_median, shell_v_median, \
    nut_h_median, nut_s_median, nut_v_median = _get_mean_med_helper(shell_pixs, shell_hsv, nut_pixs, nut_hsv)

    n = 100
    random_shell_indices = np.random.choice(len(shell_cords), size=n, replace=False)
    random_nut_indices = np.random.choice(len(nut_cords), size=n, replace=False)
    n_shell_cords = shell_cords[random_shell_indices, :]
    n_nut_cords = nut_cords[random_nut_indices, :]

    n_shell_pixs = rgb_im[n_shell_cords[:, 0], n_shell_cords[:, 1]]
    n_shell_hsv = hsv_im[n_shell_cords[:, 0], n_shell_cords[:, 1]]
    n_nut_pixs = rgb_im[n_nut_cords[:, 0], n_nut_cords[:, 1]]
    n_nut_hsv = hsv_im[n_nut_cords[:, 0], n_nut_cords[:, 1]]

    n_shell_pixs_mean, n_nut_pixs_mean, n_shell_pixs_med, n_nut_pixs_med, \
    n_shell_r_mean, n_shell_g_mean, n_shell_b_mean, \
    n_nut_r_mean, n_nut_g_mean, n_nut_b_mean, \
    n_shell_r_med, n_shell_g_med, n_shell_b_med, \
    n_nut_r_med, n_nut_g_med, n_nut_b_med, \
    n_shell_h_mean, n_shell_s_mean, n_shell_v_mean, \
    n_nut_h_mean, n_nut_s_mean, n_nut_v_mean, \
    n_shell_h_median, n_shell_s_median, n_shell_v_median, \
    n_nut_h_median, n_nut_s_median, n_nut_v_median = _get_mean_med_helper(n_shell_pixs, n_shell_hsv, n_nut_pixs, n_nut_hsv)
    
    rgb_im[shell_cords[:, 0], shell_cords[:, 1]] = np.array([0, 0, 255])
    rgb_im[nut_cords[:, 0], nut_cords[:, 1]] = np.array([255, 0, 0])
    cv2.imwrite(fname.replace("/", "_shell_nut/"), rgb_im) # actually saves shell as red because saving uses bgr

    return major_distance, minor_distance, \
    area, perimeter, \
    ellipse_area, ellipse_perimeter, \
    shell_count, nut_count, \
    shell_pixs_mean, nut_pixs_mean, shell_pixs_med, nut_pixs_med, \
    shell_r_mean, shell_g_mean, shell_b_mean, \
    nut_r_mean, nut_g_mean, nut_b_mean, \
    shell_r_med, shell_g_med, shell_b_med, \
    nut_r_med, nut_g_med, nut_b_med, \
    shell_h_mean, shell_s_mean, shell_v_mean, \
    nut_h_mean, nut_s_mean, nut_v_mean, \
    shell_h_median, shell_s_median, shell_v_median, \
    nut_h_median, nut_s_median, nut_v_median, \
    n_shell_pixs_mean, n_nut_pixs_mean, n_shell_pixs_med, n_nut_pixs_med, \
    n_shell_r_mean, n_shell_g_mean, n_shell_b_mean, \
    n_nut_r_mean, n_nut_g_mean, n_nut_b_mean, \
    n_shell_r_med, n_shell_g_med, n_shell_b_med, \
    n_nut_r_med, n_nut_g_med, n_nut_b_med, \
    n_shell_h_mean, n_shell_s_mean, n_shell_v_mean, \
    n_nut_h_mean, n_nut_s_mean, n_nut_v_mean, \
    n_shell_h_median, n_shell_s_median, n_shell_v_median, \
    n_nut_h_median, n_nut_s_median, n_nut_v_median 

def _get_mean_med_helper(shell_pixs, shell_hsv, nut_pixs, nut_hsv):
    return [shell_pixs.mean(), nut_pixs.mean(), np.median(shell_pixs), np.median(nut_pixs)] + \
list(shell_pixs.mean(axis=0)) + list(nut_pixs.mean(axis=0)) + \
list(np.median(shell_pixs, axis=0)) + list(np.median(nut_pixs, axis=0)) + \
list(shell_hsv.mean(axis=0)) + list(nut_hsv.mean(axis=0)) + \
list(np.median(shell_hsv, axis=0)) + list(np.median(nut_hsv, axis=0))

In [3]:
# test run
process_ellipse("Siirt_Pistachio/siirt (1).jpg")

(465.1001281738281,
 264.4208068847656,
 95987.5,
 1302.9818782806396,
 96589.95566267318,
 1188.495045122222,
 81805,
 14735,
 212.37640323533606,
 144.71539418617803,
 217.0,
 143.0,
 223.27443310311105,
 214.0165515555284,
 199.83822504736875,
 160.5001017984391,
 129.90023752969122,
 143.7458432304038,
 231.0,
 219.0,
 205.0,
 160.0,
 128.0,
 143.0,
 18.404608520261597,
 26.883246745308966,
 223.33353706986125,
 164.5295554801493,
 52.99192399049881,
 161.17007125890737,
 19.0,
 27.0,
 231.0,
 166.0,
 54.0,
 160.0,
 213.97666666666666,
 146.74,
 220.0,
 146.0,
 224.89,
 215.86,
 201.18,
 161.63,
 132.61,
 145.98,
 235.0,
 222.5,
 210.5,
 162.5,
 130.0,
 145.5,
 18.99,
 27.03,
 224.95,
 162.75,
 52.08,
 162.91,
 20.0,
 26.0,
 235.0,
 166.0,
 52.0,
 163.5)

In [4]:
# get raw df
df = pd.read_csv("pistachios_initial.csv", index_col=0)
df.head()

,filename,filename_camelcase,pistachio_type
0,kirmizi (23).jpg,kirmizi_23.jpg,0
1,kirmizi 21.jpg,kirmizi_21.jpg,0
2,kirmizi 35.jpg,kirmizi_35.jpg,0
3,kirmizi 475.jpg,kirmizi_475.jpg,0
4,kirmizi 313.jpg,kirmizi_313.jpg,0


In [5]:
major_axes = []
minor_axes = []
areas = []
perimeters = []
ellipse_areas = []
ellipse_perimeters = []
shell_counts = []
nut_counts = []
shell_pixs_means = []
nut_pixs_means = []
shell_pixs_meds = []
nut_pixs_meds = []
shell_r_means = []
shell_g_means = []
shell_b_means = []
nut_r_means = []
nut_g_means = []
nut_b_means = []
shell_r_meds = []
shell_g_meds = []
shell_b_meds = []
nut_r_meds = []
nut_g_meds = []
nut_b_meds = []
shell_h_means = []
shell_s_means = []
shell_v_means = []
nut_h_means = []
nut_s_means = []
nut_v_means = [] 
shell_h_medians = []
shell_s_medians = []
shell_v_medians = []
nut_h_medians = []
nut_s_medians = []
nut_v_medians = []
n_shell_pixs_means = []
n_nut_pixs_means = []
n_shell_pixs_meds = []
n_nut_pixs_meds = []
n_shell_r_means = []
n_shell_g_means = []
n_shell_b_means = []
n_nut_r_means = []
n_nut_g_means = []
n_nut_b_means = []
n_shell_r_meds = []
n_shell_g_meds = []
n_shell_b_meds = []
n_nut_r_meds = []
n_nut_g_meds = []
n_nut_b_meds = []
n_shell_h_means = []
n_shell_s_means = []
n_shell_v_means = []
n_nut_h_means = []
n_nut_s_means = []
n_nut_v_means = [] 
n_shell_h_medians = []
n_shell_s_medians = []
n_shell_v_medians = []
n_nut_h_medians = []
n_nut_s_medians = []
n_nut_v_medians = []

counter = 0
total = len(df)
for fname in df.filename.values:
    if "kirm" in fname:
        fname = "Kirmizi_Pistachio/{}".format(fname)
    else:
        fname = "Siirt_Pistachio/{}".format(fname)
    majax, minax, \
    area, perimeter, \
    ellipse_area, ellipse_perimeter, \
    shell_count, nut_count, \
    shell_pixs_mean, nut_pixs_mean, shell_pixs_med, nut_pixs_med, \
    shell_r_mean, shell_g_mean, shell_b_mean, \
    nut_r_mean, nut_g_mean, nut_b_mean, \
    shell_r_med, shell_g_med, shell_b_med, \
    nut_r_med, nut_g_med, nut_b_med, \
    shell_h_mean, shell_s_mean, shell_v_mean, \
    nut_h_mean, nut_s_mean, nut_v_mean, \
    shell_h_median, shell_s_median, shell_v_median, \
    nut_h_median, nut_s_median, nut_v_median, \
    n_shell_pixs_mean, n_nut_pixs_mean, n_shell_pixs_med, n_nut_pixs_med, \
    n_shell_r_mean, n_shell_g_mean, n_shell_b_mean, \
    n_nut_r_mean, n_nut_g_mean, n_nut_b_mean, \
    n_shell_r_med, n_shell_g_med, n_shell_b_med, \
    n_nut_r_med, n_nut_g_med, n_nut_b_med, \
    n_shell_h_mean, n_shell_s_mean, n_shell_v_mean, \
    n_nut_h_mean, n_nut_s_mean, n_nut_v_mean, \
    n_shell_h_median, n_shell_s_median, n_shell_v_median, \
    n_nut_h_median, n_nut_s_median, n_nut_v_median = process_ellipse(fname)
    major_axes.append(majax)
    minor_axes.append(minax)
    areas.append(area)
    perimeters.append(perimeter)
    ellipse_areas.append(ellipse_area)
    ellipse_perimeters.append(ellipse_perimeter)
    shell_counts.append(shell_count)
    nut_counts.append(nut_count)
    shell_pixs_means.append(shell_pixs_mean)
    nut_pixs_means.append(nut_pixs_mean)
    shell_pixs_meds.append(shell_pixs_med)
    nut_pixs_meds.append(nut_pixs_med)
    shell_r_means.append(shell_r_mean)
    shell_g_means.append(shell_g_mean)
    shell_b_means.append(shell_b_mean)
    nut_r_means.append(nut_r_mean)
    nut_g_means.append(nut_g_mean)
    nut_b_means.append(nut_b_mean)
    shell_r_meds.append(shell_r_med)
    shell_g_meds.append(shell_g_med)
    shell_b_meds.append(shell_b_med)
    nut_r_meds.append(nut_r_med)
    nut_g_meds.append(nut_g_med)
    nut_b_meds.append(nut_b_med)
    shell_h_means.append(shell_h_mean)
    shell_s_means.append(shell_s_mean)
    shell_v_means.append(shell_v_mean)
    nut_h_means.append(nut_h_mean)
    nut_s_means.append(nut_s_mean)
    nut_v_means.append(nut_v_mean) 
    shell_h_medians.append(shell_h_median)
    shell_s_medians.append(shell_s_median)
    shell_v_medians.append(shell_v_median)
    nut_h_medians.append(nut_h_median)
    nut_s_medians.append(nut_s_median)
    nut_v_medians.append(nut_v_median)
    n_shell_pixs_means.append(n_shell_pixs_mean)
    n_nut_pixs_means.append(n_nut_pixs_mean)
    n_shell_pixs_meds.append(n_shell_pixs_med)
    n_nut_pixs_meds.append(n_nut_pixs_med)
    n_shell_r_means.append(n_shell_r_mean)
    n_shell_g_means.append(n_shell_g_mean)
    n_shell_b_means.append(n_shell_b_mean)
    n_nut_r_means.append(n_nut_r_mean)
    n_nut_g_means.append(n_nut_g_mean)
    n_nut_b_means.append(n_nut_b_mean)
    n_shell_r_meds.append(n_shell_r_med)
    n_shell_g_meds.append(n_shell_g_med)
    n_shell_b_meds.append(n_shell_b_med)
    n_nut_r_meds.append(n_nut_r_med)
    n_nut_g_meds.append(n_nut_g_med)
    n_nut_b_meds.append(n_nut_b_med)
    n_shell_h_means.append(n_shell_h_mean)
    n_shell_s_means.append(n_shell_s_mean)
    n_shell_v_means.append(n_shell_v_mean)
    n_nut_h_means.append(n_nut_h_mean)
    n_nut_s_means.append(n_nut_s_mean)
    n_nut_v_means.append(n_nut_v_mean) 
    n_shell_h_medians.append(n_shell_h_median)
    n_shell_s_medians.append(n_shell_s_median)
    n_shell_v_medians.append(n_shell_v_median)
    n_nut_h_medians.append(n_nut_h_median)
    n_nut_s_medians.append(n_nut_s_median)
    n_nut_v_medians.append(n_nut_v_median)
    
    counter += 1
    if counter % 50 == 0:
        print(counter, total)
    break

In [6]:
df["major_axes"] = major_axes
df["minor_axes"] = minor_axes
df["areas"] = areas
df["perimeters"] = perimeters
df["ellipse_areas"] = ellipse_areas
df["ellipse_perimeters"] = ellipse_perimeters
df["shell_counts"] = shell_counts
df["nut_counts"] = nut_counts
df["shell_pixs_means"] = shell_pixs_means
df["nut_pixs_means"] = nut_pixs_means
df["shell_pixs_meds"] = shell_pixs_meds
df["nut_pixs_meds"] = nut_pixs_meds
df["shell_r_means"] = shell_r_means
df["shell_g_means"] = shell_g_means
df["shell_b_means"] = shell_b_means
df["nut_r_means"] = nut_r_means
df["nut_g_means"] = nut_g_means
df["nut_b_means"] = nut_b_means
df["shell_r_meds"] = shell_r_meds
df["shell_g_meds"] = shell_g_meds
df["shell_b_meds"] = shell_b_meds
df["nut_r_meds"] = nut_r_meds
df["nut_g_meds"] = nut_g_meds
df["nut_b_meds"] = nut_b_meds
df["shell_h_means"] = shell_h_means
df["shell_s_means"] = shell_s_means
df["shell_v_means"] = shell_v_means
df["nut_h_means"] = nut_h_means
df["nut_s_means"] = nut_s_means
df["nut_v_means"] = nut_v_means
df["shell_h_medians"] = shell_h_medians
df["shell_s_medians"] = shell_s_medians
df["shell_v_medians"] = shell_v_medians
df["nut_h_medians"] = nut_h_medians
df["nut_s_medians"] = nut_s_medians
df["nut_v_medians"] = nut_v_medians
df["n_shell_pixs_means"] = n_shell_pixs_means
df["n_nut_pixs_means"] = n_nut_pixs_means
df["n_shell_pixs_meds"] = n_shell_pixs_meds
df["n_nut_pixs_meds"] = n_nut_pixs_meds
df["n_shell_r_means"] = n_shell_r_means
df["n_shell_g_means"] = n_shell_g_means
df["n_shell_b_means"] = n_shell_b_means
df["n_nut_r_means"] = n_nut_r_means
df["n_nut_g_means"] = n_nut_g_means
df["n_nut_b_means"] = n_nut_b_means
df["n_shell_r_meds"] = n_shell_r_meds
df["n_shell_g_meds"] = n_shell_g_meds
df["n_shell_b_meds"] = n_shell_b_meds
df["n_nut_r_meds"] = n_nut_r_meds
df["n_nut_g_meds"] = n_nut_g_meds
df["n_nut_b_meds"] = n_nut_b_meds
df["n_shell_h_means"] = n_shell_h_means
df["n_shell_s_means"] = n_shell_s_means
df["n_shell_v_means"] = n_shell_v_means
df["n_nut_h_means"] = n_nut_h_means
df["n_nut_s_means"] = n_nut_s_means
df["n_nut_v_means"] = n_nut_v_means
df["n_shell_h_medians"] = n_shell_h_medians
df["n_shell_s_medians"] = n_shell_s_medians
df["n_shell_v_medians"] = n_shell_v_medians
df["n_nut_h_medians"] = n_nut_h_medians
df["n_nut_s_medians"] = n_nut_s_medians
df["n_nut_v_medians"] = n_nut_v_medians

In [7]:
df["ec"] = np.sqrt(1 - df["minor_axis"] ** 2/df["major_axis"] ** 2)
df["ed"] = np.sqrt(4 * df["ellipse_area"]/np.pi)
df["ex"] = df["ellipse_area"]/df["major_axis"]/df["minor_axis"]
df["k"] = df["major_axis"]/df["minor_axis"]
df["r"] = 4 * np.pi * df["ellipse_area"]/df["ellipse_perimeter"] ** 2
df["co"] = df["ed"]/df["major_axis"]
df["sf1"] = df["major_axis"]/df["ellipse_area"]
df["sf2"] = df["minor_axis"]/df["ellipse_area"]
df["sf3"] = df["ellipse_area"]/(df["major_axis"] ** 2 * 1/4 * np.pi)
df["sf4"] = df["ellipse_area"]/(df["major_axis"]/2 * df["major_axis"]/2 * np.pi)
# c convex area is kind of weird with convex hull and ex is a constant, add Deva's code later
df.head()

,Unnamed: 0,filename,filename_camelcase,pistachio_type,major_axis,minor_axis,area,perimeter,ellipse_area,ellipse_perimeter,...,ec,ed,ex,k,r,co,sf1,sf2,sf3,sf4
0,0,kirmizi (23).jpg,kirmizi_23.jpg,0,453.495544,220.994598,78732.0,1216.631593,78712.655547,1120.665639,...,0.873227,316.575529,0.785398,2.052066,0.787594,0.698079,0.005761,0.002808,0.487314,0.487314
1,1,kirmizi 21.jpg,kirmizi_21.jpg,0,464.291748,248.325897,89545.0,1268.371705,90553.003463,1169.652914,...,0.844948,339.552153,0.785398,1.869687,0.831761,0.731334,0.005127,0.002742,0.534849,0.534849
2,2,kirmizi 35.jpg,kirmizi_35.jpg,0,481.507446,219.244537,82487.0,1223.376759,82912.816940,1175.303335,...,0.890323,324.912107,0.785398,2.196212,0.754278,0.674781,0.005807,0.002644,0.455329,0.455329
3,3,kirmizi 475.jpg,kirmizi_475.jpg,0,455.230591,236.374313,83621.5,1189.217379,84512.626684,1139.466757,...,0.854628,328.031734,0.785398,1.925889,0.817953,0.720584,0.005387,0.002797,0.519241,0.519241
4,4,kirmizi 313.jpg,kirmizi_313.jpg,0,416.695648,225.598877,73160.5,1095.702659,73832.194933,1052.621353,...,0.840765,306.604094,0.785398,1.847064,0.837358,0.735799,0.005644,0.003056,0.541400,0.541400


In [8]:
df.to_csv("pistachios.csv")